## **Ingesta del archivo `pit_stops.json`**

In [ ]:
# Creamos un parametro para el nombre del archivo
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
# Creamos un parametro para la fecha del archivo
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
# Llamamos al notebook que contiene las variables de configuración
%run "../utils/configuration"

In [ ]:
# Llamamos al notebook que contiene funciones comunes
%run "../utils/common_functions"

### Paso 1 - Crear tabla **f1_raw.pit_stops**

In [ ]:
%sql
DROP TABLE IF EXISTS f1_raw.pit_stops;
CREATE EXTERNAL TABLE IF NOT EXISTS f1_raw.pit_stops(
driverId INT,
duration STRING,
lap INT,
milliseconds INT,
raceId INT,
stop INT,
time STRING)
USING JSON
OPTIONS(path "/mnt/formula1dl/raw/pit_stops.json", multiLine true)

### Paso 2 - Leer el archivo JSON

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [ ]:
pit_stops_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("stop", StringType(), True),
                                      StructField("lap", IntegerType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("duration", StringType(), True),
                                      StructField("milliseconds", IntegerType(), True)
                                     ])

In [ ]:
# El parámetro "raw_folder_path" se encuentra en el notebook "configuration"
# El parámetro "v_file_date" se encuentra en el notebook e indicamos su valor en tiempo de ejecución
pit_stops_df = spark.read \
.schema(pit_stops_schema) \
.option("multiLine", True) \
.json(f"{raw_folder_path}/pit_stops.json")
#.json(f"{raw_folder_path}/{v_file_date}/pit_stops.json")

### Paso 3 - Renombrar columnas y añadir nuevas columnas

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
pit_stops_new_df = pit_stops_df.withColumnRenamed("driverId", "driver_id") \
                               .withColumnRenamed("raceId", "race_id") \
                               .withColumn("data_source", lit(v_data_source)) \
                               .withColumn("file_date", lit(v_file_date))

In [ ]:
# La función "add_ingestion_date()" se encuentra en el notebook "common_functions"
# La fecha de ingestión será del tipo timestamp
pit_stops_with_ingestion_date_df = add_ingestion_date(pit_stops_new_df)

In [ ]:
final_df = pit_stops_with_ingestion_date_df

### Paso 4 - Escribir datos en el datalake como parquet y crear la tabla **pit_stops** en la base de datos **f1_processed**

In [ ]:
# Recordar que Databricks utiliza Hive para la gestión de tablas
# Esto quiere decir que una tabla en Hive se asocia con un directorio específico que contiene archivos de datos
# Aunque los datos subyacentes son solo archivos en un sistema de archivos, Hive proporciona una interfaz que presenta 
# estos datos como tablas.
# La base de datos tambien es un directorio. Podemos crearla manualmente, como con lenguaje SQL.
# Hive permite a los usuarios interactuar con esos datos utilizando un lenguaje similar a SQL
# En resumen, solo se esta creando un directorio y un archivo
# Escribimos el archivo con formato PARQUET en la base de datos "f1_processed" y en la tabla "pit_stops"
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.pit_stops")

In [ ]:
dbutils.notebook.exit("Success")